## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [63]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [64]:
train_data.shape

(84534, 10)

In [67]:
#focus on quadrant I and quarand II
train_data_promotion = train_data[train_data['Promotion'] == 'Yes']
train_data_promotion.shape

(42364, 10)

In [68]:
#create a new column named 'prosperous', yes = should send promotion, no = do not send promotion
train_data_promotion['prosperous'] = np.where(train_data_promotion['purchase'] == 1, 'yes', 'no')
train_data_promotion['prosperous'].value_counts()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


no     41643
yes      721
Name: prosperous, dtype: int64

In [54]:
from sklearn.model_selection import train_test_split

In [70]:
x = train_data_promotion.drop(['ID', 'Promotion', 'purchase', 'prosperous'], axis = 1)
x.head()

,V1,V2,V3,V4,V5,V6,V7
4,3,28.044332,-0.385883,1,1,2,2
8,2,31.930423,0.393317,2,3,1,2
10,1,32.770916,-1.511395,2,1,4,1
12,1,36.957009,0.133583,2,3,1,1
14,3,36.911714,-0.905350,2,2,4,1


In [71]:
y = train_data_promotion['prosperous']
y.head()

4     no
8     no
10    no
12    no
14    no
Name: prosperous, dtype: object

In [105]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [106]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [107]:
x_test[:5]

,V1,V2,V3,V4,V5,V6,V7
70569,2,25.205609,1.691984,2,2,4,1
81088,2,22.040555,0.393317,2,2,2,2
25060,0,30.947914,-1.078506,1,1,1,1
34489,2,26.066301,0.912784,2,3,3,2
29532,2,28.916179,0.393317,2,3,2,2


In [108]:
y_test[:5]

70569    no
81088    no
25060    no
34489    no
29532    no
Name: prosperous, dtype: object

In [109]:
y_pred = clf.predict(x_test[:5])
y_pred

array(['no', 'no', 'no', 'no', 'no'], dtype=object)

In [110]:
#find y_test whose value is yes
y_test.index[y_test == 'yes'].tolist()

[51885,
 17561,
 54891,
 46624,
 81799,
 66173,
 80694,
 443,
 66078,
 60034,
 14335,
 12564,
 38403,
 72905,
 21964,
 61336,
 49002,
 57877,
 28661,
 30604,
 19865,
 73952,
 79739,
 54124,
 68966,
 69203,
 20291,
 14821,
 24161,
 62468,
 57565,
 69792,
 62722,
 38357,
 74381,
 70367,
 36006,
 49468,
 60329,
 82084,
 13097,
 82570,
 73708,
 81188,
 81600,
 82956,
 51603,
 78609,
 5606,
 17861,
 28445,
 12066,
 80760,
 56327,
 79753,
 53978,
 45020,
 24710,
 73119,
 76441,
 79383,
 34447,
 55858,
 29159,
 837,
 67459,
 40530,
 38443,
 27592,
 16399,
 56904,
 29290,
 49901,
 61080,
 7909,
 60530,
 53524,
 37188,
 25402,
 64074,
 79681,
 107,
 8432,
 13338,
 32272,
 68858,
 80803,
 8526,
 6411,
 17432,
 31290,
 82469,
 15173,
 84132,
 50349,
 15618,
 5043,
 81677,
 52027,
 69513,
 48460,
 84300,
 16618,
 14121,
 24336,
 2226,
 56995,
 71525,
 56084,
 78842,
 46369,
 80633,
 76858,
 30449,
 3199,
 63303,
 56666,
 80885,
 82809,
 6633,
 68304,
 12881,
 20705,
 3812,
 64356,
 40724,
 62651,


In [111]:
#get 51885, 17561 for x_test and y_test
x_test1 = x_test.loc[[51885,17561]]
x_test1

,V1,V2,V3,V4,V5,V6,V7
51885,1,23.759391,0.826206,2,4,1,2
17561,2,26.705617,1.345672,2,3,2,2


In [112]:
y_test1 = y_test.loc[[51885,17561]]
y_test1

51885    yes
17561    yes
Name: prosperous, dtype: object

In [113]:
clf.predict(x_test1) #bad results??

array(['no', 'no'], dtype=object)

In [118]:
#try logistic regression as well
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(random_state = 42)
clf1.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [119]:
clf1.predict(x_test1) #bad results??

array(['no', 'no'], dtype=object)

In [123]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    #get quadrants I and II:
    train_data_promotion = train_data[train_data['Promotion'] == 'Yes']
    #create a new column proserous, 'True' when promotion = 1 + purchase=1, 'False' when promotion = 1 + purchase=0.
    train_data_promotion['prosperous'] = np.where(train_data_promotion['purchase'] == 1, 'yes', 'no')
    #train the RandomForest model 
    x = train_data_promotion.drop(['ID', 'Promotion', 'purchase', 'prosperous'], axis = 1)
    y = train_data_promotion['prosperous']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    #use the model to predict
    promotion = clf.predict(df)
    return promotion

In [124]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is nan.

Your nir with this strategy is 0.00.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


/home/workspace/test_results.py:9: RuntimeWarning: invalid value encountered in long_scalars
  irr = n_treat_purch / n_treat - n_ctrl_purch / n_control


(nan, 0.0)

In [125]:
def promotion_strategy1(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    #select the first and second group: 
    #group 1(promotion 'yes' + actual 'yes'), group2(promotion 'yes' + actual'no')
    train_data_promotion = train_data[train_data['Promotion'] == 'Yes']
    #create a new column proserous, 'yes' when purchase is 1, 'no' when purchase is 0.
    train_data_promotion['prosperous'] = np.where(train_data_promotion['purchase'] == 1, 'yes', 'no')
    #train the RandomForest model 
    x = train_data_promotion.drop(['ID', 'Promotion', 'purchase', 'prosperous'], axis = 1)
    y = train_data_promotion['prosperous']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
    clf = LogisticRegression()
    clf.fit(x_train, y_train)
    #use the model to predict
    promotion = clf.predict(df)
    return promotion

In [92]:
test_results(promotion_strategy1)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is nan.

Your nir with this strategy is 0.00.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


/home/workspace/test_results.py:9: RuntimeWarning: invalid value encountered in long_scalars
  irr = n_treat_purch / n_treat - n_ctrl_purch / n_control


(nan, 0.0)